In [12]:
import pandas as pd
import numpy as np

In [77]:
df = pd.read_csv(r"Z:\elton\DI\duplicates.txt")
df["frame"] += 1 

In [78]:
prev = df.iloc[0,1] - 1
cont = 0

cont_list = np.zeros((1, df.shape[0]))
bbox_matrix = np.zeros((df.shape[0], 5))

for i in df.index:
    path = f"M:\\experiment_70\\included\\{df.iloc[i, 0]}\\supporting_files\\bbox_annotations_child_face\\img_{df.iloc[i, 1]}.txt"

    try:
        bbox = np.loadtxt(path, delimiter=" ")
        bbox_matrix[i,:] = bbox[1:]
    except:
        pass


    if df.iloc[i, 1] == prev + 1: 
        cont += 1 
        
    else:
        cont = 1 
    cont_list[0, i] = cont

    prev = df.iloc[i, 1]

df["cont"] = cont_list.T

C:\Users\multimaster\AppData\Local\Temp\ipykernel_18276\2310894459.py:11: UserWarning: loadtxt: input contained no data: "M:\experiment_70\included\__20140610_16909\supporting_files\bbox_annotations_child_face\img_4328.txt"
  bbox = np.loadtxt(path, delimiter=" ")


In [79]:
df["cont"].describe()

count    327.000000
mean       7.238532
std        6.357305
min        1.000000
25%        2.000000
50%        5.000000
75%       11.000000
max       26.000000
Name: cont, dtype: float64

In [74]:
bbox_df = pd.DataFrame(bbox_matrix, columns=["xCenter","yCenter","w","h","conf"])
df_cont = pd.concat([df, bbox_df], axis=1)

In [81]:
df_cont.describe()

,frame,xCenter,yCenter,w,h,conf
count,327.000000,327.000000,327.000000,327.000000,327.000000,327.000000
mean,2702.021407,0.557320,0.696974,0.308882,0.411509,0.955378
std,2361.808591,0.169790,0.169886,0.082437,0.132318,0.060275
min,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1465.500000,0.443165,0.665433,0.312009,0.331209,0.935504
50%,1674.000000,0.565897,0.746235,0.333829,0.409817,0.962051
75%,2711.000000,0.673465,0.813923,0.355840,0.524764,0.985231
max,9311.000000,0.944238,0.939655,0.446105,0.659935,0.999686
